![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Clinical Assertion Model

In [3]:
import os

import sparknlp

print (sparknlp.version())

import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession


from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl


2.4.2

In [4]:
sparknlp_jsl

Out[3]: <module 'sparknlp_jsl' from '/databricks/python/lib/python3.7/site-packages/sparknlp_jsl/__init__.py'>

In [5]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 spark://10.164.247.156:7077 
 AppName 
 Databricks Shell

# Clinical Assertion Model (with pretrained models)

In [7]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

from sparknlp_jsl.annotator import *

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = NerDLModel.pretrained("ner_clinical", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")

# Assertion model trained on i2b2 (sampled from MIMIC) dataset
# coming from sparknlp_jsl.annotator !!
clinical_assertion = AssertionDLModel.pretrained("assertion_dl", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")
    
nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter,
    clinical_assertion
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
ner_clinical download started this may take some time.
Approximate size to download 13.8 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][ / ][OK!]

In [ ]:
%sh

wget https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed_sample_text_small.csv


In [9]:
%fs ls "file:/databricks/driver"

path,name,size
file:/databricks/driver/conf/,conf/,4096
file:/databricks/driver/ganglia/,ganglia/,4096
file:/databricks/driver/derby.log,derby.log,726
file:/databricks/driver/eventlogs/,eventlogs/,4096
file:/databricks/driver/pubmed_sample_text_small.csv,pubmed_sample_text_small.csv,9363435
file:/databricks/driver/logs/,logs/,4096


In [10]:
%fs ls "dbfs:/"

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/GMB_NER.conll,GMB_NER.conll,1663772
dbfs:/andres/,andres/,0
dbfs:/cluster-logs/,cluster-logs/,0
dbfs:/context-spell-checker/,context-spell-checker/,0
dbfs:/databricks/,databricks/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/datasets/,datasets/,0
dbfs:/ensemble_serialized/,ensemble_serialized/,0


In [11]:
dbutils.fs.cp("file:/databricks/driver/pubmed_sample_text_small.csv", "dbfs:/")

#this is already in dbfs

In [12]:
import pyspark.sql.functions as F

pubMedDF = spark.read\
                .option("header", "true")\
                .csv("pubmed_sample_text_small.csv").repartition(4)
                
pubMedDF.show(truncate=50)

+--------------------------------------------------+
 text|
+--------------------------------------------------+
Studies of the past several decades have provid...|
Neurons in the rostroventrolateral medulla (RVL...|
In a model of psychological dependence, a very ...|
To determine the functional alterations of the ...|
"""A new scheme has been described for continuo...|
OBJECTIVE: Although present treatment programs ...|
PURPOSE: To compare image quality and lesion de...|
The identification of gluten peptides eliciting...|
In two experiments, undergraduates processed a ...|
The author analyses certain aspects of the narr...|
BACKGROUND: The associations between diet and c...|
Properdin type I deficiency is characterised by...|
BACKGROUND: A transjugular intrahepatic portosy...|
AIM: To study the activation of cytotoxic T lym...|
Type II transmembrane serine proteases are an e...|
This paper describes a Japanese logographic cha...|
Immunoscintigraphy with 111In-labeled anti-CEA-...|
Arene ligand exchange in the (eta(6)-arene)Cr(C...|
The whole process of EAFP protein monoclinic cr...|
In this case report we illustrate our experienc...|
+--------------------------------------------------+
only showing top 20 rows

In [13]:
result = model.transform(pubMedDF.limit(100))

In [14]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk| assertion|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
Studies of the pa...|[[document, 0, 84...|[[document, 0, 17...|[[token, 0, 6, St...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 139, 169...|[[assertion, 139,...|
Neurons in the ro...|[[document, 0, 15...|[[document, 0, 12...|[[token, 0, 6, Ne...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 58, 81, ...|[[assertion, 58, ...|
In a model of psy...|[[document, 0, 10...|[[document, 0, 23...|[[token, 0, 1, In...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 14, 37, ...|[[assertion, 14, ...|
To determine the ...|[[document, 0, 10...|[[document, 0, 36...|[[token, 0, 1, To...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 13, 81, ...|[[assertion, 13, ...|
"""A new scheme h...|[[document, 0, 16...|[[document, 0, 10...|[[token, 0, 2, ""...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 39, 68, ...|[[assertion, 39, ...|
OBJECTIVE: Althou...|[[document, 0, 15...|[[document, 0, 20...|[[token, 0, 8, OB...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 20, 45, ...|[[assertion, 20, ...|
PURPOSE: To compa...|[[document, 0, 15...|[[document, 0, 31...|[[token, 0, 6, PU...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 38, 43, ...|[[assertion, 38, ...|
The identificatio...|[[document, 0, 17...|[[document, 0, 15...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 22, 36, ...|[[assertion, 22, ...|
In two experiment...|[[document, 0, 19...|[[document, 0, 14...|[[token, 0, 1, In...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 69, 90, ...|[[assertion, 69, ...|
The author analys...|[[document, 0, 13...|[[document, 0, 29...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 56, 103,...|[[assertion, 56, ...|
BACKGROUND: The a...|[[document, 0, 69...|[[document, 0, 13...|[[token, 0, 9, BA...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 46, 51, ...|[[assertion, 46, ...|
Properdin type I ...|[[document, 0, 10...|[[document, 0, 16...|[[token, 0, 8, Pr...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 26, P...|[[assertion, 0, 2...|
BACKGROUND: A tra...|[[document, 0, 16...|[[document, 0, 17...|[[token, 0, 9, BA...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 12, 58, ...|[[assertion, 12, ...|
AIM: To study the...|[[document, 0, 16...|[[document, 0, 26...|[[token, 0, 2, AI...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 32, 54, ...|[[assertion, 32, ...|
Type II transmemb...|[[document, 0, 16...|[[document, 0, 15...|[[token, 0, 3, Ty...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 37, T...|[[assertion, 0, 3...|
This paper descri...|[[document, 0, 74...|[[document, 0, 52...|[[token, 0, 3, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 95, 105,...|[[assertion, 95, ...|
Immunoscintigraph...|[[document, 0, 99...|[[document, 0, 20...|[[token, 0, 17, I...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 17, I...|[[assertion, 0, 1...|
Arene ligand exch...|[[document, 0, 13...|[[document, 0, 14...|[[token, 0, 4, Ar...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 98, 145,...|[[assertion, 98, ...|
The whole process...|[[document, 0, 13...|[[document, 0, 13...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 21, 43, ...|[[assertion, 21, ...|
In this case repo...|[[document, 0, 57...|[[document, 0, 15...|[[token, 0, 1, In...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 54, 88, ...|[[assertion, 54, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [15]:
result.select('text').take(3)

Out[20]: [Row(text='Studies of the past several decades have provided major insights into thestructural organization of biological membranes and mechanisms of many membranemolecular machines. However, the origin(s) of the membrane(s) and membraneproteins remains enigmatic. We discuss different concepts of the origin and earlyevolution of membranes with a focus on the evolution of the (im)permeability tocharged molecules such as proteins, nucleic acids and small ions. Reconstruction of the evolution of F-type and A/V-type membrane ATPases (ATP synthases), whichare either proton- or sodium-dependent, might help us to understand not only the origin of membrane bioenergetics but also of membranes themselves. We argue that evolution of biological membranes occurred as a process of co-evolution of lipid bilayers, membrane proteins and membrane bioenergetics.'),
 Row(text='Neurons in the rostroventrolateral medulla (RVLM) mediate baroreflex regulation(BR) of spinal sympathetic preganglionic neurons. Previously, our laboratory has shown that recovery of BR occurs in the rat after spinal hemisection. (Zahner MR,Kulikowicz E, and Schramm LP. Am J Physiol Regul Integr Comp Physiol 301:R1584-R1590, 2011). The goal of these experiments was to determine whether theobserved recovery of BR is mediated by the reorganization of ipsilateral pathwaysor by compensation by spared contralateral pathways. To determine this, weinfected the left kidney in rats with the retrograde transynaptic tracer,pseudorabies virus (PRV), either 1 or 8 wk after left spinal hemisection ateither T(3) or T(8), or after a sham lesion. In sham-lesioned rats, PRV infectionof RVLM neurons was bilateral. In all rats with a left hemisection, regardless ofthe location of the lesion (T(3) or T(8)) or postlesion recovery time (1 or 8wk), PRV infection of left RVLM neurons was significantly reduced compared withsham-lesioned rats (P < 0.05). In a separate group of rats, we performed BR testsby measuring responses of left renal sympathetic nerve activity topharmacologically induced decreases and increases in arterial pressure. In ratswith T(8) left hemisection and 8-wk recovery, BR was robust, and acute rightupper thoracic hemisection abolished all BR of left renal sympathetic nerveactivity. Collectively, these data suggest that the recovery of BR is notmediated by reorganization of ipsilateral bulbospinal connections, but instead byimproved efficacy of existing contralateral pathways.'),
 Row(text='In a model of psychological dependence, a very stable ethanol intake was induced by a chronic (1-year) intermittent (once a week) exposure to intoxicating amountsof ethanol (24 h choice between ethanol and water, followed by 2.0 g/kg i.p.).After this year, the rats had continuous access to ethanol and water. Stabilitywas shown by the ability of the rats to take the same dose of ethanol (in g/kg)when the concentration was changed from 10 to 20%. To study possible priming orinhibiting effects on ethanol intake, ethanol was injected i.p., first as 20%,40% or 60% of the intake in the 24 h prior to the injection, then as fixed doses of 0.5, 1, and 2 g/kg, and the ethanol intake during the following 24-h periodwas recorded. The results showed that, following a low dose of ethanol, voluntaryethanol intake was increased in rats with a low, and decreased in rats with ahigh, ethanol intake, while high doses of ethanol seemed to decrease voluntaryethanol intake in all rats. The results are discussed in relation to theoriesabout loss of control of drinking and relapse in humans.')]

In [16]:
result.select(F.explode(F.arrays_zip('ner_chunk.result',  'ner_chunk.begin', 'ner_chunk.end', 'ner_chunk.metadata', 'assertion.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']").alias("begin"),
        F.expr("cols['2']").alias("end"),
        F.expr("cols['3']['entity']").alias("ner_label"),
        F.expr("cols['3']['sentence']").alias("sent_id"),
        F.expr("cols['4']").alias("assertion"),).show(truncate=False)


+----------------------------------------------------------+-----+---+---------+-------+---------+
chunk |begin|end|ner_label|sent_id|assertion|
+----------------------------------------------------------+-----+---+---------+-------+---------+
many membranemolecular machines |139 |169|TREATMENT|0 |present |
nucleic acids |422 |434|TEST |5 |present |
small ions |440 |449|PROBLEM |5 |present |
A/V-type membrane ATPases |498 |522|TREATMENT|6 |present |
ATP synthases |525 |537|PROBLEM |6 |present |
whichare |541 |548|PROBLEM |6 |present |
proton |557 |562|TEST |6 |present |
sodium-dependent |568 |583|PROBLEM |6 |absent |
biological membranes |721 |740|PROBLEM |7 |present |
lipid bilayers |783 |796|TREATMENT|7 |present |
baroreflex regulation(BR |58 |81 |TEST |0 |present |
BR |183 |184|TEST |1 |present |
these experiments |346 |362|TEST |4 |present |
BR |413 |414|TEST |4 |present |
the retrograde transynaptic tracer,pseudorabies virus (PRV|585 |642|PROBLEM |5 |present |
left spinal hemisection ateither |669 |700|TREATMENT|5 |present |
a sham lesion |725 |737|PROBLEM |5 |possible |
PRV infectionof |763 |777|PROBLEM |6 |present |
a left hemisection |824 |841|PROBLEM |7 |present |
the lesion |873 |882|PROBLEM |7 |present |
+----------------------------------------------------------+-----+---+---------+-------+---------+
only showing top 20 rows

In [17]:
import pandas as pd

text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation . Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . She had close follow-up with endocrinology post discharge .
'''

print (text)

light_model = LightPipeline(model)

light_result = light_model.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]

for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
    
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)
        
df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

df

,chunks,entities,assertion
0,gestational diabetes mellitus,PROBLEM,present
1,type two diabetes mellitus,PROBLEM,present
2,"T2DM ),",PROBLEM,present
3,HTG-induced pancreatitis,PROBLEM,present
4,an acute hepatitis,PROBLEM,present
5,obesity,PROBLEM,present
6,a body mass index,PROBLEM,present
7,BMI ) of 33.5 kg/m2,PROBLEM,present
8,polyuria,PROBLEM,present
9,polydipsia,PROBLEM,present


## Writing a generic Assertion + NER function

In [19]:
from pyspark.sql.functions import monotonically_increasing_id


def get_base_pipeline (embeddings = 'embeddings_clinical'):

  documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

  # Sentence Detector annotator, processes various sentences per line

  sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

  # Tokenizer splits words in a relevant format for NLP

  tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

  # Clinical word embeddings trained on PubMED dataset
  word_embeddings = WordEmbeddingsModel.pretrained(embeddings, "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

  empty_data = spark.createDataFrame([[""]]).toDF("text")

  model = nlpPipeline.fit(empty_data)

  return model



def get_clinical_assertion (embeddings, spark_df, nrows = 100, model_name = 'ner_clinical'):

  # NER model trained on i2b2 (sampled from MIMIC) dataset
  loaded_ner_model = NerDLModel.pretrained(model_name, "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

  ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")

  # Assertion model trained on i2b2 (sampled from MIMIC) dataset
  # coming from sparknlp_jsl.annotator !!
  clinical_assertion = AssertionDLModel.pretrained("assertion_dl", "en", "clinical/models") \
      .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
      .setOutputCol("assertion")
      

  base_model = get_base_pipeline (embeddings)

  nlpPipeline = Pipeline(stages=[
      base_model,
      loaded_ner_model,
      ner_converter])

  empty_data = spark.createDataFrame([[""]]).toDF("text")

  model = nlpPipeline.fit(empty_data)

  result = model.transform(spark_df.limit(nrows))

  result = result.withColumn("id", monotonically_increasing_id())

  result_df = result.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata', 'assertion.result')).alias("cols")) \
  .select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']['entity']").alias("ner_label"),
        F.expr("cols['2']").alias("assertion"))\
        .filter("ner_label!='O'")

  return result_df

In [20]:
embeddings = 'embeddings_clinical'

model_name = 'ner_clinical'

nrows = 100

ner_df = get_clinical_assertion (embeddings, pubMedDF, nrows, model_name)

ner_df.show()

ner_clinical download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+--------------------+---------+---------+
 chunk|ner_label|assertion|
+--------------------+---------+---------+
many membranemole...|TREATMENT| present|
 nucleic acids| TEST| present|
 small ions| PROBLEM| present|
A/V-type membrane...|TREATMENT| present|
 ATP synthases| PROBLEM| present|
 whichare| PROBLEM| present|
 proton| TEST| present|
 sodium-dependent| PROBLEM| absent|
biological membranes| PROBLEM| present|
 lipid bilayers|TREATMENT| present|
baroreflex regula...| TEST| present|
 BR| TEST| present|
 these experiments| TEST| present|
 BR| TEST| present|
the retrograde tr...| PROBLEM| present|
left spinal hemis...|TREATMENT| present|
 a sham lesion| PROBLEM| possible|
 PRV infectionof| PROBLEM| present|
 a left hemisection| PROBLEM| present|
 the lesion| PROBLEM| present|
+--------------------+---------+---------+
only showing top 20 rows

In [21]:
model_name

Out[28]: 'ner_clinical'

In [22]:
%sh

cd ~ && ls -lt

total 0

In [23]:
embeddings = 'embeddings_clinical'

model_name = 'ner_posology'

nrows = 100

ner_df = get_clinical_assertion (embeddings, pubMedDF, nrows, model_name)

ner_df.show()

ner_posology download started this may take some time.
Approximate size to download 13.7 MB
[ | ][ / ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+--------------------+---------+---------+
 chunk|ner_label|assertion|
+--------------------+---------+---------+
Zahner MR,Kulikow...| DRUG| present|
intoxicating amou...| DRUG| present|
 ethanol| DRUG| present|
 2.0 g/kg| STRENGTH| present|
 ethanol| DRUG| possible|
 ethanol| DRUG| present|
 injected| FORM| present|
 i.p| ROUTE| present|
 injection| FORM| present|
 fixed doses| DOSAGE| present|
 0.5| STRENGTH| null|
 ethanol| DRUG| null|
 ethanol| DRUG| null|
 methamphetamine| DRUG| possible|
 methamphetamine| DRUG| present|
 apomorphine| DRUG| present|
 sulpiride| DRUG| possible|
 methamphetamine| DRUG| present|
 methamphetamine| DRUG| present|
 6 mg/kg| STRENGTH| present|
+--------------------+---------+---------+
only showing top 20 rows

In [24]:
embeddings = 'embeddings_clinical'

model_name = 'ner_posology'

entry_data = spark.createDataFrame([["The patient didn't take a capsule of Advil"]]).toDF("text")

ner_df = get_clinical_assertion (embeddings, entry_data, nrows, model_name)

ner_df.show()

ner_posology download started this may take some time.
Approximate size to download 13.7 MB
[ | ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+-------+---------+---------+
 chunk|ner_label|assertion|
+-------+---------+---------+
capsule| FORM| present|
 Advil| DRUG| null|
+-------+---------+---------+

In [25]:
import pandas as pd

def get_clinical_assertion_light (light_model, text):

  light_result = light_model.fullAnnotate(text)[0]

  chunks=[]
  entities=[]
  status=[]

  for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
      
      chunks.append(n.result)
      entities.append(n.metadata['entity']) 
      status.append(m.result)
          
  df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

  return df

In [26]:
clinical_text = """
Patient with severe fever and sore throat. 
He shows no stomach pain and he maintained on an epidural and PCA for pain control.
He also became short of breath with climbing a flight of stairs.
After CT, lung tumor located at the right lower lobe. Father with Alzheimer.
"""

light_model = LightPipeline(model)

get_clinical_assertion_light (light_model, clinical_text)


,chunks,entities,assertion
0,severe fever,PROBLEM,present
1,sore throat,PROBLEM,present
2,stomach pain,PROBLEM,absent
3,an epidural,TREATMENT,present
4,PCA,TREATMENT,present
5,pain control,PROBLEM,present
6,short of breath,PROBLEM,conditional
7,CT,TEST,present
8,lung tumor,PROBLEM,present
9,Alzheimer,PROBLEM,associated_with_someone_else


## Assertion with BioNLP (Cancer Genetics) NER

In [28]:
embeddings = 'embeddings_clinical'

model_name = 'ner_bionlp'

nrows = 100

ner_df = get_clinical_assertion (embeddings, pubMedDF, nrows, model_name)

ner_df.show(truncate = False)

ner_bionlp download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+---------------------------+------------------+---------+
chunk |ner_label |assertion|
+---------------------------+------------------+---------+
membranes |Cellular_component|present |
membrane |Cellular_component|present |
membranes |Cellular_component|present |
membrane ATPases |Cellular_component|present |
ATP |Simple_chemical |present |
proton |Simple_chemical |present |
membrane |Cellular_component|present |
membranes |Cellular_component|absent |
membranes |Cellular_component|present |
lipid bilayers |Cellular_component|present |
membrane |Cellular_component|null |
membrane |Cellular_component|null |
Neurons |Cell |present |
rostroventrolateral medulla|Organ |present |
RVLM |Cell |present |
spinal sympathetic |Cell |present |
neurons |Cell |present |
BR |Tissue |present |
rat |Organism |possible |
spinal |Organ |present |
+---------------------------+------------------+---------+
only showing top 20 rows

# Train your Assertion Model

In [30]:
%sh
wget https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/ner/i2b2_assertion_sample.csv

--2020-04-16 21:00:12-- https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/ner/i2b2_assertion_sample.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.77.30
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.77.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 141585 (138K) [text/csv]
Saving to: ‘i2b2_assertion_sample.csv’

 0K .......... .......... .......... .......... .......... 36% 618K 0s
 50K .......... .......... .......... .......... .......... 72% 620K 0s
 100K .......... .......... .......... ........ 100% 476K=0.2s

2020-04-16 21:00:12 (571 KB/s) - ‘i2b2_assertion_sample.csv’ saved [141585/141585]

In [31]:
dbutils.fs.cp("file:/databricks/driver/i2b2_assertion_sample.csv", "dbfs:/")


Out[39]: True

In [32]:
import pandas as pd

In [33]:

assertion_df = spark.read.option("header", True).option("inferSchema", "True").csv("i2b2_assertion_sample.csv").repartition(4)


In [34]:
assertion_df.show(3, truncate=100)

+----------------------------------------------------------------------------------------------------+--------------------------------+-------+-----+---+
 text| target| label|start|end|
+----------------------------------------------------------------------------------------------------+--------------------------------+-------+-----+---+
First , he had right lower extremity swelling and lower extremity noninvasives confirmed the diag...|a femoral deep venous thrombosis|present| 16| 20|
 # Epidural abscess : The patient was evaluated by orthopedic spine consult service and ID .| epidural abscess|present| 1| 2|
Ms Crossman is an 84 year old female with complaints of abdominal pain , diarrhea , nausea and vo...| vomiting|present| 18| 18|
+----------------------------------------------------------------------------------------------------+--------------------------------+-------+-----+---+
only showing top 3 rows

In [35]:

document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

chunk = Doc2Chunk()\
    .setInputCols("document")\
    .setOutputCol("chunk")\
    .setChunkCol("target")\
    .setStartCol("start")\
    .setStartColByTokenIndex(True)\
    .setFailOnMissing(False)\
    .setLowerCase(True)

token = Tokenizer()\
    .setInputCols(['document'])\
    .setOutputCol('token')

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["document", "token"])\
      .setOutputCol("embeddings")


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

In [36]:

assertionStatus = AssertionDLApproach()\
    .setLabelCol("label")\
    .setInputCols("document", "chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setBatchSize(128)\
    .setDropout(0.012)\
    .setLearningRate(0.015)\
    .setEpochs(1)\
    .setStartCol("start")\
    .setEndCol("end")\
    .setMaxSentLen(250)
#.setGraphFolder

clinical_assertion_pipeline = Pipeline(
    stages = [
    document,
    chunk,
    token,
    embeddings,
    assertionStatus])



In [37]:
assertion_model = clinical_assertion_pipeline.fit(assertion_df)